In [4]:
!pip install pyro-ppl

  Obtaining dependency information for pyro-ppl from https://files.pythonhosted.org/packages/f2/93/59bced321ede6eeb60061f156df8aae3f4832127fe97f4e86c567ad3b9cc/pyro_ppl-1.8.6-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.8/732.8 kB 12.3 MB/s eta 0:00:00 0:00:01


In [6]:
import numpy as np
import pandas as pd
import torch
import pyro
from pyro.distributions import Bernoulli, Normal, constraints
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer import Predictive

### Fake Data

In [7]:
# Set the random seed for reproducibility
np.random.seed(42)

# Number of data points
num_samples = 1000

# Define the true coefficients
true_coefficients = np.array([1.5, -2.0, 0.5])  # Adjust these as needed

# Generate synthetic features
X = np.random.rand(num_samples, len(true_coefficients) - 1)

# Add a column of ones for the bias term (intercept)
X = np.hstack((np.ones((num_samples, 1)), X))

# Calculate the log-odds
log_odds = X.dot(true_coefficients)

# Generate synthetic binary labels (0 or 1) based on the log-odds
probabilities = 1 / (1 + np.exp(-log_odds))
labels = np.random.binomial(n=1, p=probabilities)

# Create a DataFrame to store the data
data = pd.DataFrame(data={"target": labels})
for i in range(X.shape[1] - 1):
    data[f"feature_{i}"] = X[:, i]

### Run Model

In [8]:
# Pyro model and guide
def model(data):
    coefs = pyro.sample("coefs", Normal(0, 1).expand([data.shape[1] - 1]).to_event(1))
    logits = torch.matmul(data[:, 1:], coefs)
    with pyro.plate("data", data.shape[0]):
        pyro.sample("obs", Bernoulli(logits=logits), obs=data[:, 0])

def guide(data):
    coefs_loc = pyro.param("coefs_loc", torch.zeros(data.shape[1] - 1))
    coefs_scale = pyro.param("coefs_scale", torch.ones(data.shape[1] - 1), constraint=constraints.positive)
    pyro.sample("coefs", Normal(coefs_loc, coefs_scale).to_event(1))

# SVI and optimization
pyro.clear_param_store()
svi = SVI(model, guide, optim=Adam({"lr": 0.01}), loss=Trace_ELBO())

# Prepare the data for PyTorch
data_tensor = torch.tensor(data.values, dtype=torch.float)

In [9]:
# Training loop
num_steps = 5000
for step in range(num_steps):
    loss = svi.step(data_tensor)
    if step % 1000 == 0:
        print(f"Step {step}: loss = {loss:.2f}")

Step 0: loss = 661.86
Step 1000: loss = 616.83
Step 2000: loss = 614.50
Step 3000: loss = 619.43
Step 4000: loss = 614.67


In [11]:
# Get the parameter estimates
posterior = svi.run(data_tensor)

In [15]:
params = pyro.param("coefs_loc")
print("Estimated Coefficients:",params)


Estimated Coefficients: tensor([ 1.5731, -1.7550], requires_grad=True)


In [35]:
mape = (abs((true_coefficients[0:2] - np.array(params.detach())) / true_coefficients[0:1])*100).mean()
print(mape)

10.603292783101402
